## Date & time functions

***Formatting***
>**current_date()**: Column  
>**current_timestamp()** : Column  

>**to_timestamp(s: Column)**: Column  
>**to_timestamp(s: Column, fmt: String)**: Column  

>**date_format(dateExpr: Column, format: String)**: Column  
>**to_date(col: Column)**: Column  
>**to_date(col: Column, fmt: String)**: Column  

>**trunc(date: Column, format: String)**: Column  
>**date_trunc(format: String, timestamp: Column)**: Column

***Calculation***  

>**add_months(startDate: Column, numMonths: Int)**: Column  
>**date_add(start: Column, days: Int)**: Column  
>**date_sub(start: Column, days: Int)**: Column  
>**datediff(end: Column, start: Column)**: Column  
>**months_between(end: Column, start: Column)**: Column  
>**months_between(end: Column, start: Column, roundOff: Boolean)**: Column  
>**next_day(date: Column, dayOfWeek: String)**: Column

***Extraction***  

>**year(e: Column)**: Column  
>**quarter(e: Column)**: Column  
>**month(e: Column)**: Column  
>**dayofweek(e: Column)**: Column   
>**dayofmonth(e: Column)**: Column  
>**dayofyear(e: Column)**: Column  
>**weekofyear(e: Column)**: Column  

>**hour(e: Column)**: Column  
>**minute(e: Column)**: Column  
>**second(e: Column)**: Column  
>**last_day(e: Column)**: Column

## Unix timestamp

Unix time is a way of representing a timestamp by representing the time as the number of seconds since `January 1st, 1970 at 00:00:00 UTC`. One of the primary benefits of using Unix time is that it can be represented as an ***integer*** making it easier to parse and use across different systems.

***Unix Timestamp Functions***

>**from_unixtime(ut: Column)**: Column  
>**from_unixtime(ut: Column, f: String)**: Column  
>**unix_timestamp()**: Column  
>**unix_timestamp(s: Column)**: Column  
>**unix_timestamp(s: Column, p: String)**: Column

In [0]:
%fs ls /mnt/training/wikipedia/pageviews/pageviews_by_second.parquet/

In [0]:
SampleDF = (spark.read
  .option("inferSchema", "true") # The default, but not costly w/Parquet
  .parquet("/mnt/training/wikipedia/pageviews/pageviews_by_second.parquet/")
  .cache()                       # Cache the expensive operation
)


In [0]:
SampleDF.count() #7200000

In [0]:
SampleDF.printSchema()

Renaming the timestamp column

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
(SampleDF
 #renames while selecting the columns.. doesn't alter the SampleDF Schema
  .select( col("timestamp").alias("capturedAt"), col("site"), col("requests") )
  .printSchema()
)

In [0]:
SampleDF #DataFrame[timestamp: string, site: string, requests: int]

In [0]:
#Method 2: Using withColumnRenamed

(SampleDF
  .withColumnRenamed("timestamp", "capturedAt")
  .printSchema()
)

In [0]:
#Method 3: using toDF()

(SampleDF
  .toDF("capturedAt", "site", "requests")
  .printSchema()
)

In [0]:
tempA = (SampleDF
  .withColumnRenamed("timestamp", "capturedAt")
  .select( col("*"), unix_timestamp( col("capturedAt"), "yyyy-MM-dd HH:mm:").cast("timestamp") )
)
tempA.printSchema()

In [0]:
tempA.show()

In [0]:
pageviewsDF = (SampleDF
  .withColumnRenamed("timestamp", "capturedAt") 
  .withColumn("capturedAt", unix_timestamp( col("capturedAt"), "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp") )
)

pageviewsDF.printSchema()

root
-- capturedAt: timestamp (nullable = true)
-- site: string (nullable = true)
-- requests: integer (nullable = true)

In [0]:
pageviewsDF.show()

In [0]:
(pageviewsDF
  .select( month(col("capturedAt")).alias("month"), year(col("capturedAt")).alias("year"))
  .distinct()
  .show()                     
)

In [0]:
newDF=pageviewsDF.select("capturedAt", current_date(), current_timestamp())

In [0]:
newDF.show()

+-------------------+--------------+--------------------+
 capturedAt|current_date()| current_timestamp()|
+-------------------+--------------+--------------------+
2015-03-22 14:13:34| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 14:23:18| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 14:36:47| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 14:38:39| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 14:57:11| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 15:03:18| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 15:16:47| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 15:45:03| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 15:58:32| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:06:11| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:19:40| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:20:24| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:25:03| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:33:53| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:38:32| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:45:29| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:52:45| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 16:58:58| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 17:03:11| 2021-06-21|2021-06-21 06:05:...|
2015-03-22 17:09:24| 2021-06-21|2021-06-21 06:05:...|
+-------------------+--------------+--------------------+
only showing top 20 rows

Date calculation

In [0]:
newDF.select(months_between(current_date(), "capturedAt", True).alias("daysDifferent")).show()

+-------------+
daysDifferent|
+-------------+
 74.94862082|
 74.94840278|
 74.94810073|
 74.94805892|
 74.94764374|
 74.94750672|
 74.94720467|
 74.94657146|
 74.94626941|
 74.94609804|
 74.945796|
 74.94577957|
 74.9456754|
 74.94547752|
 74.94537336|
 74.94521767|
 74.94505488|
 74.94491562|
 74.94482116|
 74.9446819|
+-------------+
only showing top 20 rows

In [0]:
newDF.select("capturedAt",date_add("capturedAt",10)).show()

+-------------------+------------------------+
 capturedAt|date_add(capturedAt, 10)|
+-------------------+------------------------+
2015-03-22 14:13:34| 2015-04-01|
2015-03-22 14:23:18| 2015-04-01|
2015-03-22 14:36:47| 2015-04-01|
2015-03-22 14:38:39| 2015-04-01|
2015-03-22 14:57:11| 2015-04-01|
2015-03-22 15:03:18| 2015-04-01|
2015-03-22 15:16:47| 2015-04-01|
2015-03-22 15:45:03| 2015-04-01|
2015-03-22 15:58:32| 2015-04-01|
2015-03-22 16:06:11| 2015-04-01|
2015-03-22 16:19:40| 2015-04-01|
2015-03-22 16:20:24| 2015-04-01|
2015-03-22 16:25:03| 2015-04-01|
2015-03-22 16:33:53| 2015-04-01|
2015-03-22 16:38:32| 2015-04-01|
2015-03-22 16:45:29| 2015-04-01|
2015-03-22 16:52:45| 2015-04-01|
2015-03-22 16:58:58| 2015-04-01|
2015-03-22 17:03:11| 2015-04-01|
2015-03-22 17:09:24| 2015-04-01|
+-------------------+------------------------+
only showing top 20 rows